In [1]:
!pip install -q --upgrade langchain_community
!pip install tenacity


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


load web data

In [2]:
!python3 --version

Python 3.10.14


1. 3개의 블로그 포스팅 본문을 Load

In [3]:
from langchain_community.document_loaders import WebBaseLoader

urls = [
    "https://lilianweng.github.io/posts/2023-06-23-agent/",
    "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/",
    "https://lilianweng.github.io/posts/2023-10-25-adv-attack-llm/",
]

loader = WebBaseLoader(urls)

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [80]:
data = loader.load()

docs = [d.page_content for d in data]

len(docs)

3

2. 불러온 본문을 Split (Chunking) : recursive text splitter 활용

In [16]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

texts = text_splitter.create_documents(docs)
print(len(texts))
texts[0]

185


Document(page_content="LLM Powered Autonomous Agents | Lil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nAgent System Overview\n\nComponent One: Planning\n\nTask Decomposition\n\nSelf-Reflection\n\n\nComponent Two: Memory\n\nTypes of Memory\n\nMaximum Inner Product Search (MIPS)\n\n\nComponent Three: Tool Use\n\nCase Studies\n\nScientific Discovery Agent\n\nGenerative Agents Simulation\n\nProof-of-Concept Examples\n\n\nChallenges\n\nCitation\n\nReferences")

3. Chunks 를 임베딩하여 Vector store 저장: openai, chroma 사용


In [22]:
!pip install langchain-chroma
!pip install langchain_community


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [34]:
import os

# Set the API key
os.environ['OPENAI_API_KEY'] = "sk-"

In [35]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
#from langchain_community.embeddings import OllamaEmbeddings
#Open API로 바꿔야함

vectorstore = Chroma.from_documents(documents=texts, embedding=OpenAIEmbeddings())

4. User query = ‘agent memory’ 를 받아 관련된 chunks를 retrieve

In [42]:
sim_docs = vectorstore.similarity_search("agent memory")

In [43]:
print(len(sim_docs))
sim_docs[0]

4


Document(page_content='Memory stream: is a long-term memory module (external database) that records a comprehensive list of agents’ experience in natural language.\n\nEach element is an observation, an event directly provided by the agent.\n- Inter-agent communication can trigger new natural language statements.\n\n\nRetrieval model: surfaces the context to inform the agent’s behavior, according to relevance, recency and importance.\n\nRecency: recent events have higher scores\nImportance: distinguish mundane from core memories. Ask LM directly.\nRelevance: based on how related it is to the current situation / query.\n\n\nReflection mechanism: synthesizes memories into higher level inferences over time and guides the agent’s future behavior. They are higher-level summaries of past events (<- note that this is a bit different from self-reflection above)')

5. User query와 retrieved chunk 에 대해 relevance 가 있는지를 평가하는 시스템 프롬프트 작성: retrieval 퀄리티를 LLM 이 스스로 평가하도록 하고, 관련이 있으면 {‘relevance’: ‘yes’} 관련이 없으면 {‘relevance’: ‘no’} 라고 출력하도록 함. ( JsonOutputParser() 를 활용 ) - llama3 prompt format 준수

In [50]:
!pip install langchainhub

  Obtaining dependency information for langchainhub from https://files.pythonhosted.org/packages/63/af/1ede340c6283f6a70cf0e2aaa4e9c8e817ff54b0ec4df4fd956d81abffea/langchainhub-0.1.20-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests<3.0.0.0,>=2.31.0.2 from https://files.pythonhosted.org/packages/3b/b1/aad4376acab3bba3c155fdcfd8eccedf0d33b453fc6e67ca3960f3086909/types_requests-2.32.0.20240602-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [69]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import PromptTemplate


In [68]:
retriever = vectorstore.as_retriever()
llm = ChatOllama(model="llama3", temperature=0, format="json")

question = "agent memory"

searched_docs = vectorstore.similarity_search(question)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

prompt = PromptTemplate(
    template="Retrieve the relevant documents for the given query. If the document is relevant, output JSON relevance\": \"yes\". If the document is not relevant, output \"relevance\": \"no\".\n\nQuery: {question}'\n\nDocument: " + format_docs(searched_docs),
    input_variables=["question"],
    partial_variables={},
)

rag_chain = (
        {"question": RunnablePassthrough()}
        | prompt
        | llm
        | JsonOutputParser()
)

rag_chain.invoke(question)

{'relevance': 'yes'}

6. 5 에서 모든 docs에 대해 ‘no’ 라면 디버깅 (Splitter, Chunk size, overlap, embedding model, vector store, retrieval 평가 시스템 프롬프트 등) 

no 라고 하지 않음

7. 5에서 ‘yes’ 라면 질문과 명확히 관련 없는 docs 나 질문 (예: ‘I like an apple’)에 대해서는 ‘no’ 라고 나오는지 
전혀 관계 없는 쿼리인 superman movies 를 입력하면 no 출력

In [61]:
rag_chain.invoke("superman movies")

{'relevance': 'no'}

In [62]:
rag_chain.invoke("I like an apple")

{'relevance': 'no'}

8. ‘yes’ 이고 7의 평가에서도 문제가 없다면, 4의 retrieved chunk 를 가지고 답변 작성
prompt | llm | parser 형태로 작성


In [63]:
rag_chain.invoke(question)["relevance"]

'yes'

In [76]:
firstAnswerLlm = ChatOllama(model="llama3", temperature=0)

promptForFirstAnswer = PromptTemplate(
    template="Answer the question based on provided context.\n\nQuestion: {question}" + "\n\nContext: " + format_docs(searched_docs),
    input_variables=["question"],
    partial_variables={},
)

rag_chain = (
        { "question": RunnablePassthrough()}
        | promptForFirstAnswer
        | firstAnswerLlm
        | StrOutputParser()
)

firstAnswer = rag_chain.invoke(question)
firstAnswer

'Based on the context provided, I would answer the question "agent memory" as follows:\n\nIn the context of a LLM-powered autonomous agent system, agent memory refers to the ability of the agent to retain and recall information over extended periods. This is achieved through the use of both short-term and long-term memory mechanisms.\n\nShort-term memory allows the agent to learn in-context and adapt to new situations, whereas long-term memory enables the agent to retain a comprehensive list of agents\' experiences in natural language, which can be retrieved based on relevance, recency, and importance. This long-term memory module is often referred to as an external database that records observations and events directly provided by the agent.\n\nThe reflection mechanism within the agent\'s memory system synthesizes memories into higher-level inferences over time, guiding the agent\'s future behavior and decision-making processes.'

9. 생성된 답안에 Hallucination 이 있는지 평가하는 시스템 프롬프트 작성. LLM이 스스로 평가하도록 하고, hallucination 이 있으면 {‘hallucination’: ‘yes’} 없으면 {‘hallucination’: ‘no’} 라고 출력하도록 함  

In [79]:
promptForHallucinationCheck = PromptTemplate(
    template="You are LLM hallucination detector. If the answer is hallucinated, output JSON hallucination\": \"yes\". If the answer is not hallucinated, output \"hallucination\": \"no\".\n\nAnswer: {answer}",
    input_variables=["answer"],
    partial_variables={},
)

rag_chain = (
        { "answer": RunnablePassthrough()}
        | promptForHallucinationCheck
        | llm
        | JsonOutputParser()
)

rag_chain.invoke(firstAnswer)

{'hallucination': 'no'}